In [6]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import pickle
import warnings
warnings.filterwarnings('ignore')


In [4]:
# Definimos path
pathdata = '/home/daniu/Documentos/fundar/indice-mercado-trabajo-ingresos/'
pathdata = '/Users/danielarisaro/Documents/Fundar/indice-mercado-trabajo-ingresos/'
pathdata = '/home/daniufundar/Documents/Fundar/indice-mercado-trabajo-ingresos/'


In [7]:
with open(pathdata + 'modulos/diccionarios/' + 'diccionario_aglomerados.pickle', 'rb') as file:
    dict_cod_aglomerado = pickle.load(file)

with open(pathdata + 'modulos/diccionarios/' + 'diccionario_provincia.pickle', 'rb') as file:
    dict_cod_provincia = pickle.load(file)

with open(pathdata + 'modulos/diccionarios/' + 'diccionario_aglomerado_provincia.pickle', 'rb') as file:
    map_aglomerado_provincia = pickle.load(file)

with open(pathdata + 'modulos/diccionarios/' + 'diccionario_aglomerado_region.pickle', 'rb') as file:
    map_aglomerado_region = pickle.load(file)

with open(pathdata + 'modulos/diccionarios/' + 'diccionario_provincia_region.pickle', 'rb') as file:
    map_provincia_region = pickle.load(file)

In [14]:
dict_regiones_ENUT = {1: 'Gran Buenos Aires',
                    2: 'Pampeana',
                    3: 'Noroeste',
                    4: 'Noreste',
                    5: 'Cuyo',
                    6: 'Patagonia'}

In [15]:
pathfolder_ENUT =  pathdata + 'data_input/ENUT_2021/Bases de datos/'
filename = 'enut2021_base.txt'
df_enut_base = pd.read_csv(pathfolder_ENUT + filename, sep='|')


In [16]:
columnas = pd.DataFrame(df_enut_base.columns)

In [78]:
pondera = 'WPER'
df_temp = df_enut_base[df_enut_base['EDAD_SEL']>=14]
df_selected = df_temp[(~df_temp['TSS_GRANGRUPO_TNR'].isna())][['TSS_GRANGRUPO_TNR', 'SEXO_SEL', 'REGION', pondera]]
df_selected['Multiplication'] = df_selected['TSS_GRANGRUPO_TNR'] * df_selected[pondera]

numerador = df_selected.groupby(['SEXO_SEL', 'REGION'])['Multiplication'].sum().unstack(level=0)
denominador = df_selected.groupby(['SEXO_SEL', 'REGION'])[pondera].sum().unstack(level=0)
tasa = numerador.div(denominador, fill_value=np.nan)    
tasa = tasa.rename(columns= {2:'Varon', 1:'Mujer'})

size = df_temp[(~df_temp['TSS_GRANGRUPO_TNR'].isna())].groupby(['REGION', 'SEXO_SEL']).size().unstack(level=1)
size = size.rename(columns= {2:'N_v', 1:'N_m'})

n_pob = df_temp[(~df_temp['TSS_GRANGRUPO_TNR'].isna())].groupby(['REGION', 'SEXO_SEL'])[pondera].sum().unstack(level=1)
n_pob = n_pob.rename(columns= {2:'N_pob_v', 1:'N_pob_m'})

size[['N_pob_v', 'N_pob_m']] = n_pob[['N_pob_v', 'N_pob_m']]
size['N_pob_tot'] = n_pob['N_pob_v'] + n_pob['N_pob_m'] 

ratio = tasa['Varon'] / tasa['Mujer']

error = pd.DataFrame(index=size.index, columns=['LI', 'LS', 'ME', 'ER'])

ratio_minutos_promedio_no_pago = pd.concat([tasa, size, ratio*100, error], axis=1)
ratio_minutos_promedio_no_pago.rename(columns={0: 'Tiempo TNR'}, inplace=True)

ratio_minutos_promedio_no_pago.index = ratio_minutos_promedio_no_pago.index.map(dict_regiones_ENUT)

In [80]:
ratio_minutos_promedio_no_pago.to_csv(pathdata + 'data_output/indicadores/05_tiempo_no_remunerado.csv')